# Budgetbuchung Analyse: Exploration

In [1]:
%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
import qgrid
from datetime import datetime as dtt

from pa_lib.file import data_files, load_bin, store_bin, store_excel
from pa_lib.data import calc_col_partitioned, clean_up_categoricals, flatten, replace_col, cond_col, desc_col
from pa_lib.util import obj_size
from pa_lib.log  import time_log

# display long columns completely
pd.set_option('display.max_colwidth', 200)

In [2]:
data_files()

,size,mtime
name,,
bd_by_week.feather,85.6 MB,08.05.19 13:54:40
bd_data.feather,200.9 MB,03.06.19 09:01:24
bd_data_raw.feather,333.2 MB,03.06.19 09:01:14
bd_data_vkprog.feather,70.6 MB,03.06.19 09:01:26
bd_ek_minmax.feather,1.5 MB,09.05.19 08:54:54
bd_long_by_week.feather,9.2 MB,08.05.19 13:55:16
crm_data.feather,52.5 MB,03.06.19 08:42:56
crm_data_vkprog.feather,43.1 MB,03.06.19 08:42:57
plz_data.feather,135.7 KB,03.06.19 08:41:23


In [3]:
pv_data = load_bin('pv_data.feather').sort_values(['PV_NR', 'JAHR_KW']).reset_index(drop=True)
pv_info = load_bin('pv_info.feather').set_index('PV_NR')

2019-06-04 15:53:48 [INFO] Reading from file /home/pa/data/pv_data.feather
2019-06-04 15:53:48 [INFO] Finished loading binary file in 0.09s (0.16s CPU)
2019-06-04 15:53:49 [INFO] Reading from file /home/pa/data/pv_info.feather
2019-06-04 15:53:49 [INFO] Finished loading binary file in 0.01s (0.07s CPU)


# Rangliste der Verträge nach Umsatz

Nur Verträge, die seit 2017 jedes Jahr Umsatz generieren, sortiert nach Gesamtumsatz absteigend

In [4]:
desc_col(pv_info, det=True)

,DTYPE,NULLS,UNIQUE,MEM,RANGE
Titel,object,0/7851,7851,1.1 MB,"[ Bern - Bethlehemstrasse 24 [26973],zb Zentralbahn AG [32060]]"
"totalResBrutto,",float64,0/7851,7658,122.7 KB,"[0.0,200595358.35695997]"
totalResNettoNetto,float64,0/7851,7707,122.7 KB,"[0.0,128880929.3894724]"
totalAusBrutto,float64,0/7851,7828,122.7 KB,"[22.739726027393,201559729.90018842]"
totalAusNettoNetto,float64,0/7851,7838,122.7 KB,"[22.739726027393,129567091.89990559]"
partnerNr,int64,0/7851,5767,122.7 KB,"[100035,656032]"
Partner,object,0/7851,5767,823.9 KB,"[""Zürich"" Versicherungs-Gesellschaft [495776],Özdemir Ökkes [614921]]"
firstKw,int64,0/7851,258,122.7 KB,"[201401,201945]"
lastKw,int64,0/7851,332,122.7 KB,"[201401,202152]"
NettoNetto_Aus_2014,float64,0/7851,6650,122.7 KB,"[0.0,15000520.852361444]"


In [5]:
pv_liste = tuple(pv_info.loc[(pv_info.firstKw < 201501) & (pv_info.lastKw >= 201901),:]
#                        .query('Netto_Aus_2014 * Netto_Aus_2015 * Netto_Aus_2016 * Netto_Aus_2017 * Netto_Aus_2018 * Netto_Aus_2019 > 0')
#                        .query('Netto_Res_2014 * Netto_Res_2015 * Netto_Res_2016 * Netto_Res_2017 * Netto_Res_2018 * Netto_Res_2019 > 0')
                        .query('NettoNetto_Aus_2017 * NettoNetto_Aus_2018 * NettoNetto_Aus_2019 > 0')
                        .query('NettoNetto_Res_2017 * NettoNetto_Res_2018 * NettoNetto_Res_2019 > 0')
                        .eval('Sort = NettoNetto_Aus_2017 + NettoNetto_Aus_2018 + NettoNetto_Aus_2019')
                        .sort_values('Sort', ascending=False).index.values)
qgrid.show_grid(pv_info.loc[pv_liste[:20], 'Titel Partner NettoNetto_Aus_2017 NettoNetto_Aus_2018 NettoNetto_Aus_2019'.split()])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Check: Buchungen zu Top-20 Verträgen ansehen

In [6]:
pv_top20 = pv_data.loc[pv_data['PV_NR'].isin(pv_liste[:20])]
qgrid.show_grid(pv_top20.loc[(pv_top20.PV_NR==20199) & (pv_top20.JAHR=='2017')].sort_values(['JAHR', 'KW']))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
pv_top20.pivot_table(values='AUS_NETTO_NETTO', index='PV_NR', columns='JAHR', aggfunc='sum', 
                     fill_value=0, margins=True).astype('int').sort_values('All', ascending=False)

JAHR,2014,2015,2016,2017,2018,2019,2020,2021,All
PV_NR,,,,,,,,,
All,19049090,31874947,31582403,28843090,32863731,27487019,1630017,31990,173362291
20199,2328177,3135001,2970895,2894285,6500596,5247109,660,0,23076725
32006,1680107,3847266,4038754,3735374,3474999,2855370,570198,16418,20218488
11566,2237400,2939844,2725263,2759339,2781892,2082872,1878,0,15528491
32053,874983,2251806,2131305,2030363,2053926,1622937,348290,14191,11327803
26504,1267386,1848237,1739684,1447249,1637377,1645373,0,0,9585309
32049,1318039,2582534,2565712,951523,841652,882342,77236,0,9219040
25191,1416451,1737409,1624415,1548009,1464180,1139291,9744,0,8939502
309052,651895,1708910,1973343,1273895,1259508,867216,28503,0,7763272


# Aggregationen hinzufügen

#### Jahres-Summenkurven für Nettoumsatz pro Partner/Vertrag/Position

Fürs aktuelle Jahr werden die Jahressummen vom letzten Jahr benutzt. Dies impliziert, dass derselbe Umsatz erwartet wird. 
So zeigt die Summenkurve die Zielerreichung relativ zum Vorjahr.

In [8]:
def make_year_grp_sumcurve(df, year_col, grp_col, data_col, prefix=''):
    # rowmasks for this/last year
    this_year = (df[year_col] == dtt.today().year)
    last_year = (df[year_col] == dtt.today().year-1)

    # build new columns with sum/cumsum per year/grp
    df = (df
          .pipe(calc_col_partitioned, f'{prefix}sumJahr', fun='sum',    on=data_col, part_by=[year_col, grp_col])
          .pipe(calc_col_partitioned, f'{prefix}cumJahr', fun='cumsum', on=data_col, part_by=[year_col, grp_col]))

    # replace this year's sums with last year's
    last_year_sum_map = df.loc[last_year].groupby(grp_col)[data_col].agg('sum')
    df.loc[this_year, f'{prefix}sumJahr'] = df.loc[this_year, grp_col].apply(lambda x: last_year_sum_map[x])

    # divide cumsum by sum to get sum curve [0, 1], show in %
    df = (df.eval(f'{prefix}crvJahr = ({prefix}cumJahr / {prefix}sumJahr) * 100 + 0.5')
            .fillna({f'{prefix}crvJahr': 0})
            .astype({f'{prefix}crvJahr': 'int'}))
    return df

#### Auf Aggregationen

In [9]:
pv_by_week = (pv_data
              .pipe(make_year_grp_sumcurve, year_col='JAHR', grp_col='PV_NR', data_col='RES_NETTO_NETTO', prefix='res_')
              .pipe(make_year_grp_sumcurve, year_col='JAHR', grp_col='PV_NR', data_col='AUS_NETTO_NETTO', prefix='aus_'))

In [10]:
qgrid.show_grid(pv_by_week)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:
desc_col(pv_by_week, det=True)

,DTYPE,NULLS,UNIQUE,MEM,RANGE
PV_NR,int64,0/1478359,7851,11.3 MB,"[26,311503]"
JAHR_KW,int64,0/1478359,418,11.3 MB,"[201401,202152]"
JAHR,category,0/1478359,8,1.4 MB,"[2014,2021]"
KW,category,0/1478359,53,1.4 MB,"[01,53]"
RES_BRUTTO,float64,0/1478359,85293,11.3 MB,"[0.0,4089637.83883066]"
RES_NETTO_NETTO,float64,0/1478359,215067,11.3 MB,"[-25.599412008256,2566898.6604734533]"
AUS_BRUTTO,float64,0/1478359,202953,11.3 MB,"[0.692307692307,1577148.971208175]"
AUS_NETTO_NETTO,float64,0/1478359,428790,11.3 MB,"[0.409719602665,940441.8088653579]"
PV_TITEL,category,0/1478359,7851,4.0 MB,"[ Bern - Bethlehemstrasse 24 [26973],zb Zentralbahn AG [32060]]"
PARTNER_NR,int64,0/1478359,5767,11.3 MB,"[100035,656032]"


# Buchungsverlauf graphisch zeigen

Für x-Achse: 

In [17]:
pv_data['JahrKw'] = pv_data.JAHR.astype('float') + (pv_data.KW.astype('float') - 1) / 53

In [12]:
def select_Pv(df, PvNr):
    return df.loc[df.PV_NR.isin(flatten(PvNr))]

#### Alle Buchungen

In [77]:
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.transform import linear_cmap, factor_cmap

output_notebook()

pv_sel = pv_liste[:10]

p = figure(title=f"Buchungen über Aushang", x_axis_label='Datum', y_axis_label='Netto', plot_width=900)
p.circle(x='JahrKw', y='AUS_NETTO_NETTO', source=select_Pv(pv_data, pv_sel), size=6, alpha=0.6,
         color=linear_cmap('PV_NR', 'Category20_20', np.min(pv_data.PV_NR), np.max(pv_data.PV_NR)))
show(p)
p = figure(title=f"Buchungen über Reservation", x_axis_label='Datum', y_axis_label='Netto', plot_width=900)
p.circle(x='JahrKw', y='RES_NETTO_NETTO', source=select_Pv(pv_data, pv_sel), size=6, alpha=0.6,
         color=linear_cmap('PV_NR', 'Category20_20', np.min(pv_data.PV_NR), np.max(pv_data.PV_NR)))
show(p)

Loading BokehJS ...

#### Jahresverlauf vergleichen
Verträge: Top 20 ohne SBB

In [78]:
def graph_jahresverlauf(PvNr, typ='aushang'):
    """Jahres-Buchungsverlauf zeigen. PvNr kann >=1 PvNr enthalten, typ in ('aushang', 'reservation') oder Abk."""
    import altair as alt
    alt.data_transformers.enable('default', max_rows=None)

    data = select_Pv(pv_by_week, PvNr)
    if typ[:3] == 'res':
        prefix = 'res_'
    elif typ[:3] == 'aus':
        prefix = 'aus_'
    else:
        raise ValueError("typ in ('aushang', 'reservation') oder abgekürzt")

    pv_select = alt.selection_multi(fields=['PV_NR'], nearest=True)
    pv_color = alt.condition(pv_select,
                             alt.Color('PV_NR:N', legend=None),
                             alt.value('lightgray'))

    yr_select = alt.selection_multi(fields=['Jahr'])
    yr_color = alt.condition(yr_select,
                             alt.value('black'),
                             alt.value('lightgray'))

    # X axis: no auto-scaling per category
    kw_axis = alt.X('Kw', scale=alt.Scale(rangeStep=None))

    # line graphs
    lines = alt.Chart(data).mark_line(strokeWidth=3, interpolate='linear').encode(
        x=kw_axis,
        color=pv_color,
        opacity=alt.Opacity('Jahr', legend=None),
        tooltip=['Kw', 'Jahr', f'{prefix}cumJahr', f'{prefix}crvJahr']
    ).add_selection(
        pv_select
    ).transform_filter(
        pv_select
    ).transform_filter(
        yr_select
    )
    lines_cum = lines.encode(y=f'{prefix}cumJahr')
    lines_crv = lines.encode(y=f'{prefix}crvJahr')

    # clickable Pv legend
    pv_legend = alt.Chart(data).mark_rect().encode(
        y=alt.Y('PV_NR:N', sort=PV_NR, axis=alt.Axis(orient='right')),
        color=pv_color
    ).add_selection(
        pv_select
    )

    # clickable AJahr legend
    yr_legend = alt.Chart(data).mark_circle(size=150).encode(
        y=alt.Y('Jahr:N', axis=alt.Axis(orient='right')),
        color=yr_color,
        opacity=alt.condition(yr_select,
                              alt.Opacity('Jahr:N', legend=None),
                              alt.value(0.25))
    ).add_selection(
        yr_select
    )

    # lay out graphs
    return ((lines_cum | (pv_legend | yr_legend)) & 
            (lines_crv | (pv_legend | yr_legend))
           ).configure_view(height=400, width=700)

In [19]:
graph_jahresverlauf(pv_liste[:20], 'aushang').display()

ValueError: Jahr encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

VConcatChart({
  config: Config({
    view: ViewConfig({
      height: 400,
      width: 700
    })
  }),
  vconcat: [HConcatChart({
    hconcat: [Chart({
      data:           PV_NR  JAHR_KW  JAHR  KW     RES_BRUTTO  RES_NETTO_NETTO  \
      90278      5145   201403  2014  03   42143.842453     28326.561685   
      90279      5145   201404  2014  04   34710.184906     25484.419747   
      90280      5145   201405  2014  05   42178.554277     30644.294114   
      90281      5145   201406  2014  06   23496.752755     14110.538212   
      90282      5145   201407  2014  07   25021.746715     14270.026923   
      90283      5145   201408  2014  08   25146.500000     17315.450000   
      90284      5145   201409  2014  09   37295.636388     29945.202194   
      90285      5145   201410  2014  10   21127.000000     14009.821364   
      90286      5145   201411  2014  11   24076.000000     13922.608789   
      90287      5145   201412  2014  12   10508.000000      5170.537048   
   

# Aushang per Vertrag per Datum
Mit Vergleich zu Vorjahren (gleiche KW)

In [ ]:
def aushang(date):
    (jahr, kw) = date.isocalendar()[:2]
    
    result_columns = ['PvNr', 'Jahr', 'aus_sumJahr', 'aus_cumJahr', 'aus_crvJahr']
    result_labels  = ['PvNr', 'Jahr', 'total', 'cum', 'prc']
    tab = (pv_by_week.query('Kw == @kw')
                     .loc[:,result_columns]
                     .rename(columns=dict(zip(result_columns, result_labels)))
                     .pivot(index='PvNr', columns='Jahr', values=['total', 'cum', 'prc']))
    cols = [f'{lbl}_{yr}' for (lbl, yr) in tab.columns.to_flat_index()]
    tab.set_axis(labels=cols, axis='columns', inplace=True)
    return tab

### Aushang bis heute

In [ ]:
aus_per_heute = aushang(dtt.today())

alle_pv = aus_per_heute.loc[:,['total_2018', 'cum_2019', 'cum_2018', 'cum_2017']].sum(axis=0).astype('int')
print(f'Über alle Verträge:\n{alle_pv}')

aus_per_heute.query('total_2018 > 20000')

### Bereits gebuchter Aushang bis Ende Jahr

**Achtung:** Der 28. Dezember ist gemäss ISO-Logik der letzte Tag, der sicher noch in diesem Geschäftsjahr liegt

In [ ]:
aus_per_ende_jahr = aushang(dtt(dtt.today().year, month=12, day=28))

alle_pv = aus_per_ende_jahr.loc[:,['total_2018', 'cum_2019']].sum(axis=0).astype('int')
print(f'Über alle Verträge:\n{alle_pv}')

aus_per_ende_jahr.query('total_2018 > 20000')

# Reservation per Vertrag per Datum
Mit Vergleich zu Vorjahren (gleiche KW)

In [ ]:
def reservation(date):
    (jahr, kw) = date.isocalendar()[:2]
    
    result_columns = ['PvNr', 'Jahr', 'res_sumJahr', 'res_cumJahr', 'res_crvJahr']
    result_labels  = ['PvNr', 'Jahr', 'total', 'cum', 'prc']
    tab = (pv_by_week.query('Kw == @kw')
                     .loc[:,result_columns]
                     .rename(columns=dict(zip(result_columns, result_labels)))
                     .pivot(index='PvNr', columns='Jahr', values=['total', 'cum', 'prc']))
    cols = [f'{lbl}_{yr}' for (lbl, yr) in tab.columns.to_flat_index()]
    tab.set_axis(labels=cols, axis='columns', inplace=True)
    return tab

### Reservation bis heute

In [ ]:
res_per_heute = reservation(dtt.today())

alle_pv = res_per_heute.loc[:,['total_2018', 'cum_2019', 'cum_2018', 'cum_2017']].sum(axis=0).astype('int')
print(f'Über alle Verträge:\n{alle_pv}')

res_per_heute

### Berechne prozentuelle Abweichung auf tiefstes Vorjahr, Betragsabweichung (skaliert auf Vorjahr)

In [ ]:
def make_diff_cols(df):
    df = (df.assign(prc_diff = df.prc_2019 - np.minimum(df.prc_2017, df.prc_2018))
            .eval('cum_diff = prc_diff/100 * total_2018'))
    return df

In [ ]:
r = make_diff_cols(res_per_heute)

In [ ]:
r[['total_2018', 'prc_diff', 'cum_diff']].describe()

In [ ]:
r.reset_index(inplace=True)

In [ ]:
r = r.assign(Partner = r.PvNr.apply(lambda x: pv_info.at[x,'Partner']), 
             Titel   = r.PvNr.apply(lambda x: pv_info.at[x,'Titel']),
             Total   = r.total_2017 + r.total_2018 + r.total_2019)

pv_output = (pd.DataFrame(dict(Vertrag=r.PvNr, Partner=r.Partner, Titel=r.Titel, Total_2017=r.total_2017, Total_2018 = r.total_2018,
                               Stand_2017=r.cum_2017, Stand_2018=r.cum_2018, Stand_2019=r.cum_2019, Diff=r.cum_diff))
             .sort_values('Diff')
             .reset_index(drop=True))

store_excel(pv_output, 'res_per_pv.xlsx')

### Plot: % Vorjahr vs. % dieses Jahr, Grösse ~ Umsatz Vorjahr

In [ ]:
import altair as alt

# Neue Spalte 'pv_size' = srqt(Umsatz 2018)
data = (r.query('total_2018 > 1000')
         .assign(pv_size=np.sqrt(r.total_2018))
         .reset_index().sort_values('total_2018'))

points = alt.Chart(data).mark_circle(clip=True).encode(
    x=alt.X('prc_2018'),
    y=alt.Y('prc_2019', scale=alt.Scale(domain=[0, 100])),
    size='pv_size',
    tooltip=['PvNr'],
    #color='pv_size'
)

# Norm-Diagonale
diag = alt.Chart(
    pd.DataFrame({'x': [0, 100], 'y': [0, 100]})
).mark_line(color='lightgray', strokeWidth=1).encode(x='x', y='y')

(diag + points).configure_view(width=600, height=600).interactive()